# ![](https://ga-dash.s3.amazonaws.com/production/assets/logo-9f88ae6c9c3871690e33280fcf557f33.png) Webscraping Project 4 Lab

Week 4 | Day 4

In this project, we will practice two major skills: collecting data by scraping a website and then building a binary predictor with Logistic Regression.

We are going to collect salary information on data science jobs in a variety of markets. Then using the location, title and summary of the job we will attempt to predict the salary of the job. For job posting sites, this would be extraordinarily useful. While most listings DO NOT come with salary information (as you will see in this exercise), being to able extrapolate or predict the expected salaries from other listings can help guide negotiations.

Normally, we could use regression for this task; however, we will convert this problem into classification and use Logistic Regression.

- Question: Why would we want this to be a classification problem?
- Answer: While more precision may be better, there is a fair amount of natural variance in job salaries - predicting a range be may be useful.

Therefore, the first part of the assignment will be focused on scraping Indeed.com (or other sites at your team's discretion). In the second part, the focus is on using listings with salary information to build a model and predict high or low salaries and what features are predictive of that result.

### Scraping job listings from Indeed.com

We will be scraping job listings from Indeed.com using BeautifulSoup. Luckily, Indeed.com is a simple text page where we can easily find relevant entries.

First, look at the source of an Indeed.com page: (http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10")

Notice, each job listing is underneath a `div` tag with a class name of `result`. We can use BeautifulSoup to extract those. 

#### Setup a request (using `requests`) to the URL below. Use BeautifulSoup to parse the page and extract all results (HINT: Look for div tags with class name result)

In [1]:
URL = "http://www.indeed.com/jobs?q=data+scientist+$20,000&l=New+York&start=10"

In [2]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re

In [3]:
## YOUR CODE HERE

r = requests.get(URL)

In [4]:
r

<Response [200]>

In [5]:
page = r.content

In [6]:
soup = BeautifulSoup(page, 'lxml')

Let's look at one result more closely. A single `result` looks like

```
<div class=" row result" data-jk="2480d203f7e97210" data-tn-component="organicJob" id="p_2480d203f7e97210" itemscope="" itemtype="http://schema.org/JobPosting">
<h2 class="jobtitle" id="jl_2480d203f7e97210">
<a class="turnstileLink" data-tn-element="jobTitle" onmousedown="return rclk(this,jobmap[0],1);" rel="nofollow" target="_blank" title="AVP/Quantitative Analyst">AVP/Quantitative Analyst</a>
</h2>
<span class="company" itemprop="hiringOrganization" itemtype="http://schema.org/Organization">
<span itemprop="name">
<a href="/cmp/Alliancebernstein?from=SERP&amp;campaignid=serp-linkcompanyname&amp;fromjk=2480d203f7e97210&amp;jcid=b374f2a780e04789" target="_blank">
    AllianceBernstein</a></span>
</span>
<tr>
<td class="snip">
<nobr>$117,500 - $127,500 a year</nobr>
<div>
<span class="summary" itemprop="description">
C onduct quantitative and statistical research as well as portfolio management for various investment portfolios. Collaborate with Quantitative Analysts and</span>
</div>
</div>
</td>
</tr>
</table>
</div>
```

While this has some more verbose elements removed, we can see that there is some structure to the above:
- The salary is available in a `nobr` element inside of a `td` element with `class='snip`.
- The title of a job is in a link with class set to `jobtitle` and a `data-tn-element="jobTitle`.  
- The location is set in a `span` with `class='location'`. 
- The company is set in a `span` with `class='company'`. 

In [7]:
print soup.prettify()

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta content="text/html;charset=utf-8" http-equiv="content-type"/>
  <!-- pll -->
  <script src="/s/af4f8a6/en_US.js" type="text/javascript">
  </script>
  <link href="/s/6aa5893/jobsearch_all.css" rel="stylesheet" type="text/css"/>
  <link href="http://rss.indeed.com/rss?q=data+scientist+%2420%2C000&amp;l=New+York" rel="alternate" title="Data Scientist $20,000 Jobs, Employment in New York, NY" type="application/rss+xml"/>
  <link href="/m/jobs?q=data+scientist+%2420%2C000&amp;l=New+York" media="handheld" rel="alternate"/>
  <script type="text/javascript">
   window['closureReadyCallbacks'] = [];

    function call_when_jsall_loaded(cb) {
        if (window['closureReady']) {
            cb();
        } else {
            window['closureReadyCallbacks'].push(cb);
        }
    }
  </script>
  <script src="/s/954c65a/jobsearch-all-compiled.js" type="text/javascript">
  </script>
  <script type="text/javascript">
   var pingUrlsForGA = [];

var

### Write 4 functions to extract each item: location, company, job, and salary.

example: 
```python
def extract_location_from_result(result):
    return result.find ...
```


- Make sure these functions are robust and can handle cases where the data/field may not be available.
- Test the functions on the results above
- Include any other features you may want to use later (e.g. summary, #of reviews...)

In [8]:
## YOUR CODE HERE
import pandas as pd

def get_location(result):
    
    loc = []
    
    for d in soup.findAll('span', class_ = 'location'):
        loc.append(d.text)
    
    return pd.Series(loc)

In [9]:
print get_location(soup)

0                                     New York, NY
1                               New York, NY 10036
2                                     New York, NY
3                                     New York, NY
4                                     New York, NY
5                New York, NY 10001 (Chelsea area)
6                New York, NY 10111 (Midtown area)
7     New York, NY 10271 (Financial District area)
8                                     New York, NY
9                                     New York, NY
10                                    New York, NY
11                                   Manhattan, NY
12                                    New York, NY
13                               Basking Ridge, NJ
14                                  Bronxville, NY
dtype: object


In [10]:
def get_company(result):
    
    company = []
    
    for d in soup.findAll('span', class_ = 'company'):
        company.append(d.text.strip())
    return pd.Series(company)

In [11]:
print get_company(soup)

0                 Bloomberg
1                    Indeed
2                WorkFusion
3                  Barclays
4                7Park Data
5                 Dow Jones
6             Capital Group
7                     CAPCO
8                 Bloomberg
9               Yahoo! Inc.
10                 StreetID
11    DEPARTMENT OF FINANCE
12        Guidepoint Global
13                Collabera
14        Leopard Solutions
dtype: object


In [12]:
def get_job(result):
    
    job = []
    
    for d in soup.findAll(class_ = 'jobtitle'):
        job.append(d.text.strip())
    return pd.Series(job)

In [13]:
print get_job(soup)

0          Machine Learning Quantitative Analyst
1                                  Data Engineer
2                   Deep Learning Data Scientist
3                       AVP Quantitative Analyst
4                          Senior Data Scientist
5         Experiment Data Analyst / Statistician
6                          Senior Data Scientist
7                          Senior Data Scientist
8              Senior Data Scientist - Verticals
9                             Research Scientist
10                          Quantitative Analyst
11                           Policy Data Analyst
12    Data Scientist / Lead Quantitative Analyst
13                                Data Scientist
14                         Data Research Analyst
dtype: object


In [14]:
def get_salary(result):
    sal = []
    
    for d in result.findAll('td', class_ = 'snip'):
        sal.append((str(d.nobr)).strip())
    
    return pd.Series(sal)

In [15]:
print get_salary(soup)

0                                      None
1                                      None
2                                      None
3                                      None
4                                      None
5                                      None
6                                      None
7                                      None
8                                      None
9                                      None
10             <nobr>$120,000 a year</nobr>
11    <nobr>$68,239 - $80,000 a year</nobr>
12                                     None
13                                     None
14                                     None
dtype: object


Now, to scale up our scraping, we need to accumulate more results. We can do this by examining the URL above.

- "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10"

There are two query parameters here we can alter to collect more results, the `l=New+York` and the `start=10`. The first controls the location of the results (so we can try a different city). The second controls where in the results to start and gives 10 results (thus, we can keep incrementing by 10 to go further in the list).

### Complete the following code to collect results from multiple cities and starting points. 
- Indeed.com only has salary information for an estimated 20% of job postings. You may want to add other cities to the list below to gather more data. 
- Remember to convert your salary to U.S. Dollars to match the other cities if the currency is different

In [16]:
url_template = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l={&start={"

In [17]:
url_template = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l={&start={"
url_list = url_template.split('{')
max_results_per_city = 100

results = []

for city in set(['New+York', 'Chicago', 'San+Francisco', 'Austin', 'Atlanta']):
    
    for start in range(0, max_results_per_city, 10):
        
        url = url_list[0] + city + url_list[1] + str(start)
        
        r = requests.get(url)
        if str(r) == "<Response [200]>":
            
            page = r.content
            soup = BeautifulSoup(page, 'lxml')
            
            results.append(get_job(soup))
            results.append(get_location(soup))
            results.append(get_company(soup))
            results.append(get_salary(soup))
            
        else:
            print "This url is not responding favorably."
            print "url =", url
            print "response =", r
            
        # Grab the results from the request (as above)
        # Append to the full set of results

#### Use the functions you wrote above to parse out the 4 fields - location, title, company and salary. Create a dataframe from the results with those 4 columns.

In [119]:
# Initializing the dataframe

df = pd.DataFrame(results[0:4]).T
print len(df)
df.head()


15


,0,1,2,3
0,"Senior Software Engineer, Data Science","San Francisco, CA 94105",Marin Software,None
1,Save lives with machine learning,"San Francisco, CA",Blue Owl,None
2,Data Scientist/Machine Learning Engineer,"Pleasanton, CA 94588",Workday,None
3,"Machine Learning Deployment Engineer, Professi...","San Francisco, CA",Google,None
4,Data Engineer,"San Francisco, CA",Opendoor,None


In [120]:
# Number of columns scrapped
m = 4

for n in range(m, (len(results) - m), m):
    
    # Creates an interim df for each page scrapped
    df1 = pd.DataFrame(results[n:(n+m)]).T
    
    # Concatenate column-wise with df
    df = pd.concat([df,df1])

# Labelling each coloumn
df.columns = ['Job Title', 'Location', 'Company', 'Salary']

# Checking df
print len(df)
df.head()

735


,Job Title,Location,Company,Salary
0,"Senior Software Engineer, Data Science","San Francisco, CA 94105",Marin Software,None
1,Save lives with machine learning,"San Francisco, CA",Blue Owl,None
2,Data Scientist/Machine Learning Engineer,"Pleasanton, CA 94588",Workday,None
3,"Machine Learning Deployment Engineer, Professi...","San Francisco, CA",Google,None
4,Data Engineer,"San Francisco, CA",Opendoor,None


In [121]:
df['Salary'].unique()

array(['None', '<nobr>$120,000 - $150,000 a year</nobr>',
       '<nobr>$150,000 - $180,000 a year</nobr>',
       '<nobr>$4,000 a month</nobr>', '<nobr>$5,541 a month</nobr>',
       '<nobr>$4,500 - $4,900 a month</nobr>',
       '<nobr>$5,400 - $6,500 a month</nobr>',
       '<nobr>$4,599 - $6,066 a month</nobr>',
       '<nobr>$140,000 - $160,000 a year</nobr>',
       '<nobr>$4,916 a month</nobr>',
       '<nobr>$5,259 - $6,941 a month</nobr>',
       '<nobr>$3,520 - $4,600 a month</nobr>',
       '<nobr>$6,250 - $10,833 a month</nobr>',
       '<nobr>$3,616 - $5,433 a month</nobr>',
       '<nobr>$4,500 a month</nobr>',
       '<nobr>$23.44 - $29.80 an hour</nobr>',
       '<nobr>$90,000 a year</nobr>',
       '<nobr>$88,305 - $114,802 a year</nobr>',
       '<nobr>$104,349 - $135,656 a year</nobr>',
       '<nobr>$80,000 a year</nobr>',
       '<nobr>$150,000 - $205,000 a year</nobr>',
       '<nobr>$130,000 a year</nobr>',
       '<nobr>$100,000 - $125,000 a year</nobr>',
      

In [122]:
# Convert money in string form of $100,000 or $20 into float values

def conv_money(x):
    
    if ',' in x:
        x = x[1:].split(',')
        y = float(x[0] + x[1])
    else:
        y = float(x[1:])
    
    return y

In [123]:
# Find salary values

def reg_sal_low(x):
    matchObj = re.findall( r'(\$\d+\W*\d+)', x, re.I)
    if matchObj:
        return conv_money(matchObj[0])
    else:
        return 0.

In [124]:
def reg_sal_high(x):
    matchObj = re.findall( r'(\$\d+\W*\d+)', x, re.I)
    if matchObj:
        try:
            return conv_money(matchObj[1])
        except:
            return 0.
    else:
        return 0.


In [125]:
def reg_sal_freq(x):
    
    matchObj = re.findall( r'(year|month|hour|day)', x, re.I)
    if matchObj:
        return matchObj[0]
    else:
        return 'None'

In [126]:
df['Salary Lower'] = df['Salary'].apply(reg_sal_low)

In [127]:
df['Salary Higher'] = df['Salary'].apply(reg_sal_high)

In [128]:
df['Salary Frequency'] = df['Salary'].apply(reg_sal_freq)

In [129]:
df.head()

,Job Title,Location,Company,Salary,Salary Lower,Salary Higher,Salary Frequency
0,"Senior Software Engineer, Data Science","San Francisco, CA 94105",Marin Software,None,0.0,0.0,None
1,Save lives with machine learning,"San Francisco, CA",Blue Owl,None,0.0,0.0,None
2,Data Scientist/Machine Learning Engineer,"Pleasanton, CA 94588",Workday,None,0.0,0.0,None
3,"Machine Learning Deployment Engineer, Professi...","San Francisco, CA",Google,None,0.0,0.0,None
4,Data Engineer,"San Francisco, CA",Opendoor,None,0.0,0.0,None


In [117]:
df['Salary Lower'].unique()

array([  0.00000000e+00,   1.20000000e+05,   1.50000000e+05,
         4.00000000e+03,   5.54100000e+03,   4.50000000e+03,
         5.40000000e+03,   4.59900000e+03,   1.40000000e+05,
         4.91600000e+03,   5.25900000e+03,   3.52000000e+03,
         6.25000000e+03,   3.61600000e+03,   2.34400000e+01,
         9.00000000e+04,   8.83050000e+04,   1.04349000e+05,
         8.00000000e+04,   1.30000000e+05,   1.00000000e+05,
         3.00000000e+04,   5.17060000e+04,   6.82390000e+04,
         7.02860000e+04,   4.00000000e+04,   1.25000000e+05,
         7.86300000e+04,   2.00000000e+05,   5.79690000e+04,
         9.32570000e+04,   1.04674000e+05,   1.80000000e+05,
         1.10000000e+05,   2.00000000e+01])

In [141]:
# Scrapping the web for a list of best cities for data scientists to work

url = 'https://infogr.am/30f00c18-997e-4a9d-b903-191899a53890'

In [142]:
d = requests.get(url)

In [143]:
d

<Response [200]>

In [144]:
page1 = d.content

In [145]:
soup1 = BeautifulSoup(page1, 'lxml')

In [152]:
print soup1.prettify()

<!DOCTYPE HTML>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width,initial-scale=1" name="viewport"/>
  <title>
   Data Science Jobs - Infogram, charts &amp; infographics
  </title>
  <script>
   // If infographic has been embedded without 'src' parameter redirect with 'src' parameter
        if (window !== window.top) {
            window.location.href = document.location.pathname + '?src=embed';
        }
  </script>
  <meta content="346759272005624" property="fb:app_id"/>
  <meta content="https://s3-eu-west-1.amazonaws.com/infogram-thumbs-1024/41ba4d20-2a1f-4115-bb8f-9302fd0b5645.jpg" property="og:image"/>
  <meta content="image/jpeg" property="og:image:type"/>
  <meta content="1024" property="og:image:width"/>
  <meta content="Data Science Jobs - Infogram, charts &amp; infographics" property="og:title"/>
  <meta content="" property="og:description"/>
  <meta content="https://infogr.am/30f00c18-997e-4a9d-b903-191899a53890" property="og:url"/>
  <l

In [156]:
for d in soup1.findAll('td'):
    print d.text

Lastly, we need to clean up salary data. 
1. Some of the salaries are not yearly but hourly, these will be useful to us for now
2. The salaries are given as text and usually with ranges.

#### Filter out the salaries that are not yearly (filter those that refer to hour)

In [22]:
## YOUR CODE HERE

#### Write a function that takes a salary string and converts it to a number, averaging a salary range if necessary

In [23]:
## YOUR CODE HERE

### Save your results as a CSV

In [24]:
## YOUR CODE HERE